In [6]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import os
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime,date, timedelta
import shutil
from time import time
from ahmedsabri import *



In [8]:
# reading data
df=pd.read_csv('Unit 02 Macro Database.csv',header=None)

In [9]:
# creating column name
df.columns=df.iloc[0:4].fillna('').astype(str).apply(' '.join).str.strip()
df.columns=df.columns.str.replace(" ","_")
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace("-","_")
df=df.iloc[5:,:].reset_index(drop=True)

In [10]:
# remove duplicated column
df=remove_duplicated_columns(df,False)

In [11]:
df=convert_string_to_nan(df)

In [12]:
# dropping columns and rows containing 85 % nan values
df=keep_rows(keep_columns(df,0.85),0.85)

In [13]:
# converting all data to float except the first date column
df=to_float(df,1)

In [14]:
df['description_units_sheet_tag_no._parameter']=pd.to_datetime(df['description_units_sheet_tag_no._parameter'])

In [16]:
X=df.drop(['description_units_sheet_tag_no._parameter','charge_heater_bridge_wall_temp._°c_03_ti_015_pv'],axis=1)
y=df['charge_heater_bridge_wall_temp._°c_03_ti_015_pv']

In [17]:
X.fillna(X.median(),inplace=True)

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)
lir=RandomForestRegressor()
lir.fit(X_train,y_train)


RandomForestRegressor()

In [21]:
y_pred=lir.predict(X_test)

In [22]:
truth=pd.DataFrame()
truth['y_actual']= y_test
truth['Y_predict']= y_pred
truth['difference']=np.abs(truth['Y_predict']-truth['y_actual'])

In [23]:
r2_score(y_test,y_pred)

0.9966594707473435

In [24]:
from sklearn.metrics import mean_absolute_error     

In [25]:
mean_absolute_error(y_test,y_pred)

3.578194578797392

In [26]:
truth.difference.mean()

3.5781945787973877

In [27]:
feat_impo=pd.DataFrame()
feat_impo['column']=X.columns
feat_impo['importance']= lir.feature_importances_
feat_impo.sort_values(by= 'importance', ascending=False)

,column,importance
92,u_2/3_stack_temperature_°c_03_ti_014_pv,0.831670
136,03_h_01_skin__03_txi_017_b_pv,0.128822
53,splitter_reflux_flow__03_fic_004_daca.lastgoodpv,0.014621
82,fuel_gas_from_u_39_nm3/hr_02_fi_058_daca.lastg...,0.006196
91,flue_gas_oxygen_analyser_from_03_h_01_%_03_ai_...,0.003431
...,...,...
74,splitter_overhead_pressure_bar(g)_03_pic_006_d...,0.000020
50,stripper_overhead_pressure_#_c_bar(g)_02_pi_02...,0.000019
48,stripper_overhead_pressure_#a_bar(g)_02_pi_023...,0.000017
49,stripper_overhead_pressure_#b_bar(g)_02_pi_023...,0.000012
